##  Cillium Agents

In [ ]:
!kubectl -n kube-system get pods -l k8s-app=cilium -owide

## Show Production App

In [ ]:
!kubectl get pods -n kube-system 

In [11]:
!kubectl get pods -n production -owide 

NAME                          READY   STATUS    RESTARTS        AGE   IP             NODE                 NOMINATED NODE   READINESS GATES
collector-7dbcfbc46b-9fsqm    1/1     Running   1 (3h29m ago)   8h    10.244.2.81    my-cluster-worker2   <none>           <none>
mgmt-7d7f75d999-fptbz         1/1     Running   1 (3h29m ago)   8h    10.244.2.179   my-cluster-worker2   <none>           <none>
mock-mongo-779c79f69f-tdgpf   1/1     Running   1 (3h29m ago)   8h    10.244.2.57    my-cluster-worker2   <none>           <none>
ui-5bc5f74697-jfxbb           1/1     Running   1 (3h29m ago)   8h    10.244.2.240   my-cluster-worker2   <none>           <none>


In [ ]:
!kubectl get pod -n production  --show-labels

## Show Endpoints of UI 

In [ ]:
#%%bash 
# Endpoints in node "my-cluster-worker"
# kubectl -n kube-system exec cilium-zh6rx  -- cilium-dbg endpoint list  

# Endpoints in node "my-cluster-worker2"
!kubectl -n kube-system exec cilium-8mgsf -- cilium-dbg endpoint list


## Show Hubble UI 


1. Show **production** namespace
2. Click on UI pod and view the identity=8389 and labels
3. show the traffic to UI 

## Hubble Observe

### Output Examples

In [ ]:
# network traffic in production namespace 
#!hubble observe --namespace production 

# network ingress/egress traffic of ui ONLY  
!hubble observe --pod production/ui-5b9668859f-fs6fq 

# network ingress traffic to UI 
#!hubble observe --pod production/ui-655869d49c-wtkdt   --traffic-direction ingress

### Explain LOG

```
Sep 11 08:25:07.258: production/mgmt-75f6bf4f9c-4qr76:51308 (ID:7056) -> production/ui-655869d49c-wtkdt:8080 (ID:8389) to-overlay FORWARDED (TCP Flags: SYN)
Sep 11 08:25:07.258: production/mgmt-75f6bf4f9c-4qr76:51308 (ID:7056) -> production/ui-655869d49c-wtkdt:8080 (ID:8389) to-endpoint FORWARDED (TCP Flags: SYN)
```

#### to-overlay VS to-endpoint
1. **to-endpoint**: Indicates direct pod-to-pod communication without encapsulation or routing through an overlay network.  
   *This typically happens when the source and destination pods are on the same node or within the same network and do not require encapsulation.*

2. **to-overlay**: Indicates that the traffic is being encapsulated and forwarded through an overlay network,  
   usually, the source and destination pods are on different nodes, and encapsulation is needed to route the traffic between them.


#### Example to-endpoint

The collector and the ui pods are running on the same node "my-cluster-worker"

In [12]:
!hubble observe --to-pod production/ui-5bc5f74697-jfxbb --from-pod production/mgmt-7d7f75d999-fptbz 

Sep 16 06:31:40.565: production/ui-5bc5f74697-jfxbb:44478 (ID:16030) <- production/mgmt-7d7f75d999-fptbz:8080 (ID:61457) to-endpoint FORWARDED (TCP Flags: ACK, PSH)
Sep 16 06:31:40.566: production/ui-5bc5f74697-jfxbb:44478 (ID:16030) <- production/mgmt-7d7f75d999-fptbz:8080 (ID:61457) to-endpoint FORWARDED (TCP Flags: ACK, FIN)
Sep 16 06:31:42.134: production/mgmt-7d7f75d999-fptbz:60104 (ID:61457) -> production/ui-5bc5f74697-jfxbb:8080 (ID:16030) to-endpoint FORWARDED (TCP Flags: SYN)
Sep 16 06:31:42.134: production/mgmt-7d7f75d999-fptbz:60104 (ID:61457) -> production/ui-5bc5f74697-jfxbb:8080 (ID:16030) to-endpoint FORWARDED (TCP Flags: ACK)
Sep 16 06:31:42.134: production/mgmt-7d7f75d999-fptbz:60104 (ID:61457) -> production/ui-5bc5f74697-jfxbb:8080 (ID:16030) to-endpoint FORWARDED (TCP Flags: ACK, PSH)
Sep 16 06:31:42.136: production/mgmt-7d7f75d999-fptbz:60104 (ID:61457) -> production/ui-5bc5f74697-jfxbb:8080 (ID:16030) to-endpoint FORWARDED (TCP Flags: ACK, FIN)
Sep 16 06:31:42.136:

In [13]:
!hubble observe --verdict DROPPED 

#### Example to-overlay

The mgmt and the ui pods are running on the same node "my-cluster-worker"

### Detailed Network LOG (JSON Format)

Show a single LOG and review the JSON content:
1. source and dst : IPs, labels
2. Node Labels 

In [15]:
!hubble observe --pod production/ui-5bc5f74697-jfxbb   --last 1 --output json | jq 

{
  "flow": {
    "time": "2024-09-16T06:32:20.612896770Z",
    "uuid": "b3e50ecc-383e-45ad-9b0f-2b9f60205ae7",
    "verdict": "FORWARDED",
    "ethernet": {
      "source": "ea:3e:ed:32:43:0f",
      "destination": "66:12:9b:64:49:36"
    },
    "IP": {
      "source": "10.244.1.227",
      "destination": "10.244.2.240",
      "ipVersion": "IPv4"
    },
    "l4": {
      "UDP": {
        "source_port": 53,
        "destination_port": 50858
      }
    },
    "source": {
      "ID": 627,
      "identity": 16152,
      "namespace": "kube-system",
      "labels": [
        "k8s:io.cilium.k8s.namespace.labels.kubernetes.io/metadata.name=kube-system",
        "k8s:io.cilium.k8s.policy.cluster=default",
        "k8s:io.cilium.k8s.policy.serviceaccount=coredns",
        "k8s:io.kubernetes.pod.namespace=kube-system",
        "k8s:k8s-app=kube-dns"
      ],
      "pod_name": "coredns-76f75df574-72z2v",
      "workloads": [
        {
          "name": "coredns",
          "kind": "Deployment"
 

### [TODO: What is **trace_observation_point**]

## Enforcement

### Isolate qa and production

In [ ]:
#!kubectl apply -f policies/isolate-prod-qa.yaml

In [ ]:
#!hubble observe --from-pod production/ui-655869d49c-wtkdt  --verdict DROPPED

# Helm Update

In [ ]:
!kubectl apply -f ringfence/production 
!kubectl apply -f ringfence/qa
!kubectl apply -f ringfence/dev

In [ ]:
!kubectl get cnp -n qa

In [ ]:
!kubectl delete -f ringfence/production 
!kubectl delete -f ringfence/qa
!kubectl delete -f ringfence/dev